In [74]:
import sys
sys.prefix
from pprint import pprint

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov 14 12:05:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  | 00000000:06:00.0 Off |                  N/A |
| 34%   47C    P8              17W / 250W |     27MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 67.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [12]:
# # Install required libraries
# !pip install datasets==2.10.1
# !pip install transformers==4.28.0
# !pip install accelerate
# !pip install jiwer
# !pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: datasets
    Found existing installation: datasets 1.18.3
    Uninstalling datasets-1.18.3:
      Successfully uninstalled datasets-1.18.3


In [5]:
# Import libraries
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer
from huggingface_hub import notebook_login

In [6]:
notebook_login()

map columns for noisy data

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file into a DataFrame
df = pd.read_csv('data/sentences_noise_data_sample.csv')

# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create a DataFrame for the training set
train_subset = pd.DataFrame({
    'Sentence': train_df['Sentence'],
    'noisy_sentence': train_df.apply(lambda row: row[row['Best_Alternative']], axis=1)
})

# Create a DataFrame for the testing set
test_subset = pd.DataFrame({
    'Sentence': test_df['Sentence'],
    'noisy_sentence': test_df.apply(lambda row: row[row['Best_Alternative']], axis=1)
})

# Save the training and testing sets to new CSV files
train_subset.to_csv('data/train_sentences_noise_data.csv', index=False)
test_subset.to_csv('data/test_sentences_noise_data.csv', index=False)


convert to json

In [8]:
import pandas as pd

def convert_csv_to_json(csv_path, json_output_path):

    df = pd.read_csv(csv_path)

    json_data = df.to_json(orient='records')


    with open(json_output_path, 'w') as json_file:
        json_file.write(json_data)

csv_file_path = 'data/train_sentences_noise_data.csv'
json_output_file_path = 'data/train_output.json'
convert_csv_to_json(csv_file_path, json_output_file_path)

In [9]:
csv_file_path = 'data/test_sentences_noise_data.csv'
json_output_file_path = 'data/test_output.json'
convert_csv_to_json(csv_file_path, json_output_file_path)

In [10]:
# Load the dataset from the JSON files
dataset = load_dataset('json', data_files={'train': 'data/train_output.json',
                                           'test': 'data/test_output.json'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jinda/.cache/huggingface/datasets/json/default-1f260b0f7599e2c4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Sentence', 'noisy_sentence'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['Sentence', 'noisy_sentence'],
        num_rows: 2000
    })
})


In [12]:
print(dataset['train'][:5])
print(dataset['test'][:5])

{'Sentence': ["It's not easy to win.", "I don't want you to tell Tom about what happened here.", "She's in the National Theatre Company.", 'You should distinguish between right and wrong.', 'I knew that Tom was a coward.'], 'noisy_sentence': ['itznot  e azyt ow in', 'idom twamdy oud o telltom  ap outwhat  happenedhe re', 's hesi nthenatiomar d heatrec onpa my', 'distinku is hbedwee mligh dandwro mg  youwer eb robab', 'ikm ow dhaddom i s outraged ikn ew dhad tomw asac o']}
{'Sentence': ['Tom sat in the dentist’s chair.', "He didn't say what kind of bike he has.", "I'd be happy to come in.", 'Good afternoon! How are you?', "I told you I didn't want this."], 'noisy_sentence': ['dond oogou this false t eeth', 'cy cre heditn t s ayw h adki mdov bikehe haz', 'i t be happ ytocom eim', 'hello a r eyous dilldhele', 'idolty ou  ititn   dwamddhis']}


In [13]:
# Split the dataset into train and validation sets
train_dataset, val_dataset = train_test_split(dataset['train'], test_size=0.1, random_state=42)

In [14]:
from datasets import Dataset

train_data = Dataset.from_dict(train_dataset)  # Convert the train data to a dataset
val_data = Dataset.from_dict(val_dataset)      # Convert the validation data to a dataset

In [15]:
print(type(train_data))
print(type(val_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [16]:
# Check column names in train dataset
print(train_data.column_names)

# Check column names in validation dataset
print(val_data.column_names)

['Sentence', 'noisy_sentence']
['Sentence', 'noisy_sentence']


In [17]:
print(train_data['Sentence'][:5])
print(train_data['noisy_sentence'][:5])
#print(val_data[:5])

["Do you know Tom's full name?", "Today I went to the dentist's.", "You'll ask Tom for permission to do that, won't you?", 'Morocco got into debt.', "The reason that Tom isn't here is because he's sick in bed."]
['do youk now to msvur  lnam e', 'yiwe mt tothete ndists  inee dtoothbas de', 'i n su red omd hatw  ouldtod h ati vyouazgethim to', 'mor o c cogodintod ebt', 'tom izi nbe dbecaus ehezn odfeerin g werl thereaz omthatt om iz mdherei zpe  cau z ehessi ']


In [18]:
# Define the source and target language columns
source_lang = 'noisy_sentence'
target_lang = 'Sentence'

In [19]:
print(source_lang)

noisy_sentence


In [20]:
# Define the max_length for padding and truncation
max_length = 512

In [21]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs


In [22]:
# Select a random data point from the train dataset
sample_data = train_data[0]

# Call the preprocess function on the sample data
processed_data = preprocess_function(sample_data)

# Inspect the output
print(processed_data)

{'input_ids': tensor([[    0,  2362, 26351,  1215, 19530,  4086,    35,   385,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1021,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1437,  1437,     2],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1423,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1021,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1717,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,   449,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1437,  1437,     2],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,   295,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1021,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,   885,     2,     1],
        [    0,  2362, 26351,  1215, 19530,  4086,    35,  1437,  1437,     2],
        [    0,  2362, 263

In [23]:
#train_data = preprocess_function(train_data)
#val_data = preprocess_function(val_data)

# Apply preprocess_function to train_data and val_data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [24]:
# Access a few samples from train_data
for i in range(5):
    sample_input_ids = train_data['input_ids'][i]
    sample_attention_mask = train_data['attention_mask'][i]
    sample_labels = train_data['labels'][i]

    print(f"Sample {i+1}:")
    print("Input IDs:", sample_input_ids)
    print("Attention Mask:", sample_attention_mask)
    print("Labels:", sample_labels)
    print()

Sample 1:
Input IDs: [0, 2362, 26351, 1215, 19530, 4086, 35, 109, 47, 330, 122, 7, 43601, 705, 710, 1437, 784, 8697, 364, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Labels: [0, 8275, 47, 216, 1560, 18, 455, 766, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [25]:
# Prepare DataLoader for training and validation
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

In [26]:
# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch

In [36]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="spell_correction_tatoeba",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
# verify passing the correct inputs to the trainer
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)
print("Tokenizer:", tokenizer)
print("Training Arguments:", training_args)

Train Dataset: Dataset({
    features: ['Sentence', 'noisy_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7200
})
Validation Dataset: Dataset({
    features: ['Sentence', 'noisy_sentence', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 800
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=3,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find

In [38]:
# model is initialized with the BARTForConditionalGeneration class and moved to the GPU if available.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

loading configuration file config.json from cache at /home/jinda/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "L

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [39]:
torch.cuda.empty_cache()

In [40]:
# The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/home/jinda/Desktop/torgo/torgo_inference/Machine Translation/spell_correction_tatoeba is already a clone of https://huggingface.co/matrixcc/spell_correction_tatoeba. Make sure you pull the latest changes with `repo.git_pull()`.
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Sentence, noisy_sentence. If Sentence, noisy_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/home/jinda/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7200
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distrib

  0%|          | 0/9000 [00:00<?, ?it/s]

/home/jinda/anaconda3/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Sentence, noisy_sentence. If Sentence, noisy_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24


  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.5821688771247864, 'eval_runtime': 8.3946, 'eval_samples_per_second': 95.299, 'eval_steps_per_second': 4.05, 'epoch': 1.0}


...

Model weights saved in spell_correction_tatoeba/checkpoint-9000/pytorch_model.bin
tokenizer config file saved in spell_correction_tatoeba/checkpoint-9000/tokenizer_config.json
Special tokens file saved in spell_correction_tatoeba/checkpoint-9000/special_tokens_map.json
Deleting older checkpoint [spell_correction_tatoeba/checkpoint-7500] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Sentence, noisy_sentence. If Sentence, noisy_sentence are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
/home/jinda/anaconda3/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dime

  0%|          | 0/34 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.34302985668182373, 'eval_runtime': 8.4142, 'eval_samples_per_second': 95.077, 'eval_steps_per_second': 4.041, 'epoch': 30.0}
{'train_runtime': 8761.2214, 'train_samples_per_second': 24.654, 'train_steps_per_second': 1.027, 'train_loss': 0.3115828611585829, 'epoch': 30.0}


TrainOutput(global_step=9000, training_loss=0.3115828611585829, metrics={'train_runtime': 8761.2214, 'train_samples_per_second': 24.654, 'train_steps_per_second': 1.027, 'train_loss': 0.3115828611585829, 'epoch': 30.0})

In [41]:
# Load the trained model from Hugging Face
model = BartForConditionalGeneration.from_pretrained("matrixcc/spell_correction_tatoeba")

loading configuration file config.json from cache at /home/jinda/.cache/huggingface/hub/models--matrixcc--spell_correction_tatoeba/snapshots/dfa215158f3be6b71823ac1fd20a6104d9fd99da/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0":

loading weights file pytorch_model.bin from cache at /home/jinda/.cache/huggingface/hub/models--matrixcc--spell_correction_tatoeba/snapshots/dfa215158f3be6b71823ac1fd20a6104d9fd99da/pytorch_model.bin
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at matrixcc/spell_correction_tatoeba.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
Generation config file not found, using a generation config created from the model config.


In [42]:
# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
#model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [43]:
test_dataset = dataset['test']

In [69]:
pprint(test_dataset['Sentence'][-10:])
pprint(test_dataset['noisy_sentence'][-10:])

['Please just get out of my way.',
 'A set of benefits was once in effect.',
 'If you walk a lot, you will develop calluses on your heel.',
 'The movie was better than I thought it was going to be.',
 'Sami realized Layla was dead.',
 'Tom said that he wished that Mary would eat healthier food.',
 'That is a mistake.',
 "I know it's very difficult to quit smoking.",
 'Ziri gave a talk about the Berber language.',
 "It's common in the East."]
['z dopgan gimku po nme',
 'a s ed ovb enefitzw a sonce inevve ct',
 'ivyouw alg aro tyouwilldefe lopc ar  l usesom yo ulhee r',
 'th efi lmwaz  nodwh  atiha  tthoughd itwas',
 'samirear i zetlayla waste ad',
 'edettoeadhealth ie lvood t omsai dt ha thewishetdhadm a rywoulde ath  eal',
 'ha disam i zdake  tha',
 'i d isimboz sibr eforhim dogif eupzn o king',
 'zir ispe aksper be larltheti ne',
 'thatrarel yhappen z']


In [44]:
print(len(test_dataset))

2000


In [65]:
# Inspect column names
column_names = test_dataset.column_names
print("Column names:", column_names)

# Inspect column data types
for column in column_names:
    column_data = test_dataset[column]
    data_type = type(column_data[0]) if len(column_data) > 0 else "Unknown"
    print(f"Column name: {column}, Data type: {data_type}")

Column names: ['Sentence', 'noisy_sentence']
Column name: Sentence, Data type: <class 'str'>
Column name: noisy_sentence, Data type: <class 'str'>


In [62]:
# verification
# Test the model on speaker dataset
verify_test_dataset = dataset['test']
print(len(verify_test_dataset))
print((verify_test_dataset))

2000
Dataset({
    features: ['Sentence', 'noisy_sentence'],
    num_rows: 2000
})


In [63]:
# Print the samples
cnt=0
for sample in test_dataset:
    print("input:", sample["noisy_sentence"])
    print("actual:", sample["Sentence"])
    
    cnt+=1
    if cnt==20:
        break

input: dond oogou this false t eeth
actual: Tom sat in the dentist’s chair.
input: cy cre heditn t s ayw h adki mdov bikehe haz
actual: He didn't say what kind of bike he has.
input: i t be happ ytocom eim
actual: I'd be happy to come in.
input: hello a r eyous dilldhele
actual: Good afternoon! How are you?
input: idolty ou  ititn   dwamddhis
actual: I told you I didn't want this.
input: otasi n gl ec reatureo ne alth hasmo leol lezs rig ht tobehe re  iv dheleisli febe yom deardh itis prob
actual: Not a single creature on earth has more or less right to be here.
input: wher ed iddomg  otocol
actual: Where did Tom go to college?
input: ay do mhadall ta ytof  imiz hd
actual: Tom had all day to finish doing that.
input: us inga progranma br et hern oz tadyouca madjusd dhet im e syout urnomtheheatingolair co nd ition imkacco rdimgd oaprezedschetule 
actual: Using a programmable thermostat, you can adjust the times you turn on the heating or air-conditioning according to a pre-set schedule.

In [48]:
test_data = test_dataset.map(preprocess_function, batched=True, batch_size=len(test_dataset))
model.eval()

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [49]:
predictions = []
references = []

for example in test_dataset:
    # This creates an input text by combining the values of the 'speaker' and 'prediction' fields from the current example.
    # It assumes that the example is a dictionary-like object with keys 'speaker' and 'prediction'.
    input_text = f"{example['noisy_sentence']}"

    # This creates a context where no gradients are computed, which can improve efficiency during inference.
    with torch.no_grad():

        # This tokenizes the input_text using the tokenizer and converts it into input IDs as a PyTorch tensor.
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # The generate method generates the output sequence based on the provided input.
        outputs = model.generate(input_ids=input_ids, max_length=max_length)

    predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

    references.append(example['Sentence'])
    predictions.append(predicted_sentence)

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

In [50]:
# Verify that the number of predictions and references are the same
if len(predictions) == len(references):
    print("Number of predictions and references are the same.")
else:
    print("Mismatch in the number of predictions and references.")

Number of predictions and references are the same.


In [51]:
# Print the number of predictions and references
print("Number of predictions:", len(predictions))
print("Number of references:", len(references))

# print the length of the dataset
print("Number of rows in dataset:", len(test_dataset))

Number of predictions: 2000
Number of references: 2000
Number of rows in dataset: 2000


In [75]:
# verification
print("prediction:", predictions[:10])
print("reference:", references[:10])

AttributeError: 'list' object has no attribute 'write'

In [53]:
# calculate WER

from jiwer import wer

wer_value = wer(predictions, references)
wer_percentage = wer_value * 100

print(f"WER: {wer_percentage:.2f}%")

WER: 89.53%
